In [34]:
import pandas as pd
import numpy as np
import re

In [25]:
df = pd.read_csv("./all.csv")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63923 entries, 0 to 63922
Data columns (total 12 columns):
职位        63923 non-null object
公司名称      63923 non-null object
公司地点      63923 non-null object
公司类别      63870 non-null object
薪资        62079 non-null object
学历要求      63923 non-null object
工作经验      63923 non-null object
公司规模      62287 non-null object
公司福利      52267 non-null object
发布时间      63923 non-null object
公司信息链接    63923 non-null object
分类        63923 non-null object
dtypes: object(12)
memory usage: 5.9+ MB
None


In [26]:
df = df.dropna(subset=["薪资","公司类别"])
df.to_csv('./all_salary.csv', encoding='utf_8_sig')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62031 entries, 0 to 63917
Data columns (total 12 columns):
职位        62031 non-null object
公司名称      62031 non-null object
公司地点      62031 non-null object
公司类别      62031 non-null object
薪资        62031 non-null object
学历要求      62031 non-null object
工作经验      62031 non-null object
公司规模      60459 non-null object
公司福利      51131 non-null object
发布时间      62031 non-null object
公司信息链接    62031 non-null object
分类        62031 non-null object
dtypes: object(12)
memory usage: 6.2+ MB
None


In [33]:
df = pd.read_csv("./all_salary.csv")
company_xinzhi = list(df["公司类别"].unique())
print(company_xinzhi)
onehot_leibie = pd.get_dummies(df["公司类别"], prefix ='公司类别')
print(onehot_leibie.head(5))
df = pd.concat([df,onehot_leibie],axis=1)
df = df.drop("公司类别", axis=1)
df.to_csv('./all_leibie.csv', encoding='utf_8_sig')
print(df.head(5))

['外资（欧美）', '民营公司', '上市公司', '合资', '外资（非欧美）', '创业公司', '国企', '事业单位', '非营利组织', '外企代表处', '政府机关']
   公司类别_上市公司  公司类别_事业单位  公司类别_创业公司  公司类别_合资  公司类别_国企  公司类别_外企代表处  公司类别_外资（欧美）  \
0          0          0          0        0        0           0            1   
1          0          0          0        0        0           0            0   
2          0          0          0        0        0           0            1   
3          1          0          0        0        0           0            0   
4          0          0          0        0        0           0            0   

   公司类别_外资（非欧美）  公司类别_政府机关  公司类别_民营公司  公司类别_非营利组织  
0             0          0          0           0  
1             0          0          1           0  
2             0          0          0           0  
3             0          0          0           0  
4             0          0          1           0  
   Unnamed: 0                           职位            公司名称     公司地点        薪资  \
0           0   Staff FPGA D

In [43]:
df = pd.read_csv("./all_leibie.csv")
minimum = []
maximum = []
average = []
for i in range(len(df['薪资'])):
    tmp = df['薪资'][i]
    if re.compile('年').search(tmp):
        yearTag = 1 # year salary
    elif re.compile('天').search(tmp):
        yearTag = 300 # day salary
    else:
        yearTag = 12 # month salary

    if re.compile('千').search(tmp):
        multiTag = 1000 # count as 1000
    elif re.compile('元').search(tmp):
        multiTag = 1 # count as 1
    else:
        multiTag = 10000 # count as 10000

    numberIndex = max(tmp.find("千"), tmp.find("元"), tmp.find("万"))
    numberString = tmp[0:numberIndex]
    numberList = numberString.split("-")
    if len(numberList) == 1:
        minimum.append(int(float(numberList[0]) * multiTag * yearTag))
        maximum.append(int(float(numberList[0]) * multiTag * yearTag))
        average.append(int(float(numberList[0]) * multiTag * yearTag))
    else:
        minimum.append(int(float(numberList[0]) * multiTag * yearTag))
        maximum.append(int(float(numberList[1]) * multiTag * yearTag))
        average.append(int((float(numberList[0])+float(numberList[1]))/2 * multiTag * yearTag))
# append the maxinum and minimum salary in newDF
df.insert(loc=len(df.columns), column="薪资_最大值", value=maximum)
df.insert(loc=len(df.columns), column="薪资_最小值", value=minimum)
df.insert(loc=len(df.columns), column="薪资_平均值", value=average)
df = df.drop("薪资", axis=1)
print(df.head(5))
df.to_csv('./all_salary.csv', encoding='utf_8_sig')

   Unnamed: 0  Unnamed: 0.1                           职位            公司名称  \
0           0             0   Staff FPGA Design Engineer  迈威迩科技（Marvell）   
1           1             1                    手机部-EMC专家      小米通讯技术有限公司   
2           2             2                 新产品导入工程师-NPI            西部数据   
3           3             4                       PCB工程师   北京新雷能科技股份有限公司   
4           4             5  Field Service Engineer（Dep）   上海科文斯集成微电有限公司   

      公司地点        薪资 学历要求    工作经验        公司规模  \
0  上海-浦东新区    4-5万/月   本科  5-7年经验   500-1000人   
1   上海-徐汇区  1.5-3万/月   本科  3-4年经验    10000人以上   
2   上海-闵行区  2-2.5万/月   本科  5-7年经验  1000-5000人   
3  上海-浦东新区  1-1.5万/月   大专    无需经验   500-1000人   
4       上海  1.5-2万/月   本科    2年经验    150-500人   

                                             公司福利   ...   公司类别_国企 公司类别_外企代表处  \
0                 五险一金 餐饮补贴 股票期权 弹性工作 年终奖金 补充医疗保险   ...         0          0   
1                        带薪年假 五险一金 绩效奖金 专业培训 节日福利   ...         0          0   
2  五险一金 

In [60]:
df = pd.read_csv("./all_salary.csv")
print(list(df["学历要求"].unique()))
newframe = []
for i in range(len(df['学历要求'])):
    tmp = df['学历要求'][i]
    if tmp == '初中及以下':
        newframe.append(1)
    elif tmp == '中技':
        newframe.append(2)
    elif tmp == '中专':
        newframe.append(3)
    elif tmp == '高中':
        newframe.append(4)
    elif tmp == '大专':
        newframe.append(5)
    elif tmp == '本科':
        newframe.append(6)
    elif tmp == '硕士':
        newframe.append(7)
    elif tmp == '博士':
        newframe.append(8)
    else:
        newframe.append(3)
df.drop("学历要求",axis=1)
df.insert(loc=len(df.columns), column="学历", value=newframe)
print(list(df["学历"].unique()))
df.to_csv('./all_edu.csv', encoding='utf_8_sig')

['本科', '大专', '招2人', '硕士', 'companysize_text":"50-150人', '招若干人', 'companysize_text":"10000人以上', '高中', '中专', '招3人', '中技', 'companysize_text":"150-500人', '博士', '招1人', 'companysize_text":"少于50人', '初中及以下', 'companysize_text":"500-1000人', 'companysize_text":"1000-5000人', '招5人', '招4人', '招14人', '招10人', 'companysize_text":"5000-10000人', '招20人', '招15人', '招11人', '招7人', '招30人', '招55人', '招50人', '招9人', '招6人', '招100人', '招8人', '招25人', '招13人', '招66人', '招99人', '招31人', '招22人', '招12人', '招40人', '招39人', 'companysize_text":"', '招49人', '招59人', '招16人']
[6, 5, 3, 7, 4, 2, 8, 1]


In [91]:
df = pd.read_csv("./all_edu.csv")
newframe = []
print(list(df["工作经验"].unique()))
for i in range(len(df['工作经验'])):
    tmp = df['工作经验'][i]
    if re.compile("3-4年经验").search(tmp):
        newframe.append(3.5)
    elif re.compile("5-7年经验").search(tmp):
        newframe.append(6)
    elif re.compile("8-9年经验").search(tmp):
        newframe.append(8.5)
    elif re.compile("10年以上经验").search(tmp):
        newframe.append(10)
    elif re.compile("1年经验").search(tmp):
        newframe.append(1)
    elif re.compile("2年经验").search(tmp):
        newframe.append(2)
    elif re.compile('在校生/应届生').search(tmp):
        newframe.append(0)
    elif re.compile('无需经验').search(tmp):
        newframe.append(0)
    else:
        newframe.append(0)
df = df.drop("工作经验",axis=1)
df.insert(loc=len(df.columns), column="工作经验", value=newframe)
print(list(df["工作经验"].unique()))
df.to_csv('./all_experience.csv', encoding='utf_8_sig')

['5-7年经验', '3-4年经验', '无需经验', '2年经验', '1年经验', '本科', '8-9年经验', '招1人', '招若干人', '大专', '10年以上经验', '招2人', '硕士', '博士', '招5人', '在校生/应届生', '招3人', '招10人', '中专', '初中及以下', '招6人', '招4人', '招8人', '高中', '招7人', '招20人', '中技']
[6.0, 3.5, 0.0, 2.0, 1.0, 8.5, 10.0]


In [102]:
df = pd.read_csv("./all_experience.csv")
newframe = []
print(list(df["公司规模"].unique()))
for i in range(len(df['公司规模'])):
    tmp = df['公司规模'][i]
    if re.compile("少于50").search(tmp):
        newframe.append(25)
    elif re.compile("50-150").search(tmp):
        newframe.append(100)
    elif re.compile("150-500").search(tmp):
        newframe.append(325)
    elif re.compile("500-1000").search(tmp):
        newframe.append(750)
    elif re.compile("1000-5000").search(tmp):
        newframe.append(3000)
    elif re.compile("5000-10000").search(tmp):
        newframe.append(7500)
    elif re.compile('10000人以上').search(tmp):
        newframe.append(10000)
    else:
        newframe.append(25)
df = df.drop("公司规模",axis=1)
df.insert(loc=len(df.columns), column="公司规模", value=newframe)
print(list(df["公司规模"].unique()))
df.to_csv('./all_guimo.csv', encoding='utf_8_sig')

['500-1000人', '10000人以上', '1000-5000人', '150-500人', '50-150人', '少于50人', nan, '5000-10000人', '少于50人","adid":""},{"type":"engine_search_result","jt":"0","tags":,"ad_track":"","jobid":"125346331","coid":"4418758","effect":"1","is_special_job":"","job_href":"https://jobs.51job.com/hangzhou-yhq/125346331.html?s=sou_sou_soulb&t=0","job_name":"LED显示屏（大屏）项目经理","job_title":"LED显示屏（大屏）项目经理","company_href":"https://jobs.51job.com/all/co4418758.html","company_name":"杭州巴科光电科技有限公司","providesalary_text":"0.8-1万/月","workarea":"080207","workarea_text":"异地招聘","updatedate":"05-17","iscommunicate":"","companytype_text":"民营公司","degreefrom":"","workyear":"5","issuedate":"2021-05-17 11:56:12","isFromXyz":"","isIntern":"","jobwelf":"包住","jobwelf_list":"包住","attribute_text":"异地招聘","3-4年经验","招2人","companysize_text":"少于50人']
500-1000人
10000人以上
1000-5000人
500-1000人
150-500人
50-150人
50-150人
少于50人
50-150人
150-500人
少于50人
50-150人
150-500人
少于50人
50-150人
150-500人
150-500人
1000-5000人
少于50人
少于50人
少于50人
少于50人
少于50人
150-50

TypeError: expected string or bytes-like object

In [103]:
df = pd.read_csv("./all_experience.csv")
newframe = []
print(list(df["公司规模"].unique()))
df["公司规模"] = df["公司规模"].fillna(value="少于50")
for i in range(len(df['公司规模'])):
    tmp = df['公司规模'][i]
    if re.compile("少于50").search(tmp):
        newframe.append(25)
    elif re.compile("50-150").search(tmp):
        newframe.append(100)
    elif re.compile("150-500").search(tmp):
        newframe.append(325)
    elif re.compile("500-1000").search(tmp):
        newframe.append(750)
    elif re.compile("1000-5000").search(tmp):
        newframe.append(3000)
    elif re.compile("5000-10000").search(tmp):
        newframe.append(7500)
    elif re.compile('10000人以上').search(tmp):
        newframe.append(10000)
    else:
        newframe.append(25)
df = df.drop("公司规模",axis=1)
df.insert(loc=len(df.columns), column="公司规模", value=newframe)
print(list(df["公司规模"].unique()))
df.to_csv('./all_guimo.csv', encoding='utf_8_sig')

['500-1000人', '10000人以上', '1000-5000人', '150-500人', '50-150人', '少于50人', nan, '5000-10000人', '少于50人","adid":""},{"type":"engine_search_result","jt":"0","tags":,"ad_track":"","jobid":"125346331","coid":"4418758","effect":"1","is_special_job":"","job_href":"https://jobs.51job.com/hangzhou-yhq/125346331.html?s=sou_sou_soulb&t=0","job_name":"LED显示屏（大屏）项目经理","job_title":"LED显示屏（大屏）项目经理","company_href":"https://jobs.51job.com/all/co4418758.html","company_name":"杭州巴科光电科技有限公司","providesalary_text":"0.8-1万/月","workarea":"080207","workarea_text":"异地招聘","updatedate":"05-17","iscommunicate":"","companytype_text":"民营公司","degreefrom":"","workyear":"5","issuedate":"2021-05-17 11:56:12","isFromXyz":"","isIntern":"","jobwelf":"包住","jobwelf_list":"包住","attribute_text":"异地招聘","3-4年经验","招2人","companysize_text":"少于50人']
[750, 10000, 3000, 325, 100, 25, 7500]


In [109]:
df = pd.read_csv("./all_guimo.csv")
newframe = []
print(list(df["岗位地区"].unique()))

onehot_leibie = pd.get_dummies(df["岗位地区"])
print(onehot_leibie.head(5))
df = pd.concat([df,onehot_leibie],axis=1)
df = df.drop("岗位地区", axis=1)

df.to_csv('./all_diqu.csv', encoding='utf_8_sig')

['上海-浦东新区', '上海-徐汇区', '上海-闵行区', '上海', '上海-嘉定区', '上海-松江区', '异地招聘', '上海-杨浦区', '上海-宝山区', '上海-静安区', '上海-黄浦区', '上海-青浦区', '上海-长宁区', '上海-奉贤区', '上海-普陀区', '上海-金山区', '上海-虹口区', '上海-崇明区']
   _上海  _上海-嘉定区  _上海-奉贤区  _上海-宝山区  _上海-崇明区  _上海-徐汇区  _上海-普陀区  _上海-杨浦区  \
0    0        0        0        0        0        0        0        0   
1    0        0        0        0        0        1        0        0   
2    0        0        0        0        0        0        0        0   
3    0        0        0        0        0        0        0        0   
4    1        0        0        0        0        0        0        0   

   _上海-松江区  _上海-浦东新区  _上海-虹口区  _上海-金山区  _上海-长宁区  _上海-闵行区  _上海-青浦区  _上海-静安区  \
0        0         1        0        0        0        0        0        0   
1        0         0        0        0        0        0        0        0   
2        0         0        0        0        0        1        0        0   
3        0         1        0        0        0        0        0        

In [111]:
df = pd.read_csv("./all_diqu.csv")
print(list(df["分类"].unique()))


onehot_leibie = pd.get_dummies(df["分类"])
print(onehot_leibie.head(5))
df = pd.concat([df,onehot_leibie],axis=1)
df = df.drop("分类", axis=1)

df.to_csv('./all_fenlei.csv', encoding='utf_8_sig')

['半导体/芯片', '财务/会计', '餐饮服务', '电子商务', '后端开发', '建筑设计施工', '教育行业', '律师行业', '汽车设计制作', '前端开发', '生产运营', '市场营销', '销售管理', '游戏开发', '制药医疗器械']
   制药医疗器械  前端开发  半导体/芯片  后端开发  市场营销  建筑设计施工  律师行业  教育行业  汽车设计制作  游戏开发  生产运营  \
0       0     0       1     0     0       0     0     0       0     0     0   
1       0     0       1     0     0       0     0     0       0     0     0   
2       0     0       1     0     0       0     0     0       0     0     0   
3       0     0       1     0     0       0     0     0       0     0     0   
4       0     0       1     0     0       0     0     0       0     0     0   

   电子商务  财务/会计  销售管理  餐饮服务  
0     0      0     0     0  
1     0      0     0     0  
2     0      0     0     0  
3     0      0     0     0  
4     0      0     0     0  


In [118]:
df = pd.read_csv("./all_fenlei.csv")
s = {}
t = {}
for i in range(len(df['公司福利'])):
    tmp = df["公司福利"][i]
    words = str(tmp).split(' ')
    for j in words:
        if j in s.keys():
            s[j]=s[j]+1
        else:
            s[j]=1
for i in s.keys():
    if s[i]>2000:
        t[i] = s[i]
print(t)

{'五险一金': 46437, '餐饮补贴': 19488, '股票期权': 3028, '弹性工作': 11153, '年终奖金': 27246, '补充医疗保险': 7118, '带薪年假': 14525, '绩效奖金': 28726, '专业培训': 22069, '节日福利': 12440, '免费班车': 5185, '定期体检': 20321, '交通补贴': 12529, '通讯补贴': 12129, '周末双休': 7991, '做五休二': 7163, 'nan': 10900, '员工旅游': 18060, '包吃': 2459, '补充公积金': 2604, '出国机会': 3537, '包住宿': 2011, '全勤奖': 3478, '高温补贴': 2784, '加班补贴': 2447}


In [119]:
del t["nan"]
print(t)

{'五险一金': 46437, '餐饮补贴': 19488, '股票期权': 3028, '弹性工作': 11153, '年终奖金': 27246, '补充医疗保险': 7118, '带薪年假': 14525, '绩效奖金': 28726, '专业培训': 22069, '节日福利': 12440, '免费班车': 5185, '定期体检': 20321, '交通补贴': 12529, '通讯补贴': 12129, '周末双休': 7991, '做五休二': 7163, '员工旅游': 18060, '包吃': 2459, '补充公积金': 2604, '出国机会': 3537, '包住宿': 2011, '全勤奖': 3478, '高温补贴': 2784, '加班补贴': 2447}


In [123]:
df["公司福利"] = df["公司福利"].fillna(value=0)
for i in t.keys():
    newframe = []
    for j in range(len(df['公司福利'])):
        tmp = df["公司福利"][j]
        if re.compile(i).search(str(tmp)):
            newframe.append(1)
        else:
            newframe.append(0)
    df.insert(loc=len(df.columns), column=i, value=newframe)
print(df.info())
df.to_csv('./all_fuli.csv', encoding='utf_8_sig')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62031 entries, 0 to 62030
Data columns (total 79 columns):
Unnamed: 0      62031 non-null int64
Unnamed: 0.1    62031 non-null int64
职位              62031 non-null object
公司名称            62031 non-null object
公司福利            62031 non-null object
上市公司            62031 non-null int64
事业单位            62031 non-null int64
创业公司            62031 non-null int64
合资              62031 non-null int64
国企              62031 non-null int64
外企代表处           62031 non-null int64
外资（欧美）          62031 non-null int64
外资（非欧美）         62031 non-null int64
政府机关            62031 non-null int64
民营公司            62031 non-null int64
非营利组织           62031 non-null int64
薪资_最大值          62031 non-null int64
薪资_最小值          62031 non-null int64
薪资_平均值          62031 non-null int64
学历要求            62031 non-null int64
工作经验            62031 non-null float64
公司规模            62031 non-null int64
上海              62031 non-null int64
嘉定区             62031 non-null int6

In [128]:
def get_all_substrings(input_string):
  length = len(input_string)
  return [input_string[i:j + 1] for i in range(length) for j in range(i,length)]

In [129]:
df = pd.read_csv("./all_fuli.csv")
s = {}
t = {}
for i in range(len(df['职位'])):
    tmp = df["职位"][i]
    words = get_all_substrings(str(tmp))
    for j in words:
        if j in s.keys():
            s[j]=s[j]+1
        else:
            s[j]=1
for i in s.keys():
    if s[i]>2000:
        t[i] = s[i]
print(t)

{'S': 2719, 't': 4835, 'a': 8670, ' ': 10530, 'P': 2619, 'A': 3659, 'D': 2138, 'e': 9665, 's': 3017, 'i': 6269, 'g': 3208, 'n': 7454, 'E': 3268, 'in': 2009, 'er': 2719, 'r': 5519, '-': 3605, 'M': 2537, 'C': 3433, '专': 6294, '产': 3028, '工': 19082, '工程': 17025, '工程师': 16069, '程': 18724, '程师': 16072, '师': 22660, 'l': 2475, 'c': 2767, '（': 8355, '）': 8438, '设': 3831, '设计': 3511, '计': 6193, '/': 6920, '端': 2829, '师（': 2749, '电': 4405, '件': 2206, '经': 9417, '经理': 9212, '理': 13731, '场': 4282, '售': 5149, '前': 2526, ')': 2211, 'o': 3949, '高': 3090, '高级': 2298, '级': 2940, '(': 2302, '运': 4463, 'J': 2719, '0': 2782, '销': 5747, '销售': 4771, '开': 6398, '开发': 6311, '开发工': 5408, '开发工程': 5408, '开发工程师': 5404, '发': 7270, '发工': 5761, '发工程': 5759, '发工程师': 5755, '前端': 2265, '1': 2165, '市': 3865, '市场': 3546, '总': 2359, '生': 3082, '主': 4905, '主管': 4018, '管': 5407, '务': 6225, '员': 9657, '助': 2654, '助理': 2503, '端开': 2060, '端开发': 2059, '专员': 5424, '项': 2027, '项目': 2023, '目': 2051, '商': 3471, '运营': 4185, '营': 538

In [130]:
t = {'工程师': 16069,'设计': 3511,'经理': 9212, '高级': 2298,'销售': 4771,'开发': 6311, '前端': 2265,'市场': 3546, '总': 2359, '主管': 4018,'助理': 2503, '专员': 5424, '项目': 2023, '运营': 4185,'电商': 2396, '法务': 2182}

In [131]:
for i in t.keys():
    newframe = []
    for j in range(len(df['职位'])):
        tmp = df["职位"][j]
        if re.compile(i).search(str(tmp)):
            newframe.append(1)
        else:
            newframe.append(0)
    df.insert(loc=len(df.columns), column=i, value=newframe)
print(df.info())
df.to_csv('./all_zhiwei.csv', encoding='utf_8_sig')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62031 entries, 0 to 62030
Data columns (total 92 columns):
职位         62031 non-null object
公司名称       62031 non-null object
上市公司       62031 non-null int64
事业单位       62031 non-null int64
创业公司       62031 non-null int64
合资         62031 non-null int64
国企         62031 non-null int64
外企代表处      62031 non-null int64
外资（欧美）     62031 non-null int64
外资（非欧美）    62031 non-null int64
政府机关       62031 non-null int64
民营公司       62031 non-null int64
非营利组织      62031 non-null int64
薪资_最大值     62031 non-null int64
薪资_最小值     62031 non-null int64
薪资_平均值     62031 non-null int64
学历要求       62031 non-null int64
工作经验       62031 non-null float64
公司规模       62031 non-null int64
上海         62031 non-null int64
嘉定区        62031 non-null int64
奉贤区        62031 non-null int64
宝山区        62031 non-null int64
崇明区        62031 non-null int64
徐汇区        62031 non-null int64
普陀区        62031 non-null int64
杨浦区        62031 non-null int64
松江区        62031 non-nu